<a href="https://colab.research.google.com/github/Small-Molecule-Discovery-Center/smdc_preprocess_data/blob/main/preprocess_pampa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preprocess PAMPA data
### How to use this notebook:
1. Click folder icon on left side
2. Drag PAMPA file into folder space
3. Right click on PAMPA file and hit copy path
4. Paste file path into second cell
5. Edit source_plate and donor_well_conc variables if necessary
3. Run all the cells (shift+enter or click the arrows)
7. DOWNLOAD the results_processed file - it will not get saved to your system it will just be deleted if you close the tab

In [2]:
! git clone https://github.com/Small-Molecule-Discovery-Center/smdc_preprocess_data.git
import sys
sys.path.append('/content/smdc_preprocess_data')
import preprocess_pampa as pp

Cloning into 'smdc_preprocess_data'...
remote: Enumerating objects: 58, done.
remote: Counting objects: 100% (58/58), done.
remote: Compressing objects: 100% (48/48), done.
remote: Total 58 (delta 25), reused 29 (delta 8), pack-reused 0
Receiving objects: 100% (58/58), 66.09 KiB | 4.72 MiB/s, done.
Resolving deltas: 100% (25/25), done.


### Variables to fill out for each plate run

In [1]:
# path to file
infile="/content/smdc_preprocess_data/data/pampa_explorer_example_results.xlsx"

# source plate barcode: if multiple, separate with semicolons like "OTPARP1-MA01;OTPARP2-MA01"
source_plate="SMDC123"

# concentration in uM
donor_well_conc=50

### Run code

In [3]:
pp.preprocess_pampa(infile=infile, source_plate=source_plate, donor_well_conc=donor_well_conc)

Destination plate: pampa_explorer_example
Unique samples pre: 33
Unique samples post: 32
Data saved to ./pampa_explorer_example_processed.csv
